In [2]:
import cv2
import mediapipe as mp
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np

# Load the trained model
model = joblib.load('model_traind_R_Federer.joblib')

# Pose estimation setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# New video path
new_video_path = r'teste_model\t1.mp4'



# Define the landmark indices and names
landmark_indices = [mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.RIGHT_SHOULDER,
                    mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST,
                    mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW,
                    mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_HIP,
                    mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE,
                    mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE,
                    mp_pose.PoseLandmark.LEFT_ANKLE]

landmark_names = [landmark.name for landmark in landmark_indices]
# Output video file for replicating the skeleton
output_video_filename = 'landmarks_replicated_video.mp4'
output_video_writer = cv2.VideoWriter(output_video_filename, cv2.VideoWriter_fourcc(*'mp4v'), 30, (1280, 720))

# Output CSV file for the new data
csv_new_filename = 'data_compare.csv'
csv_new_file = open(csv_new_filename, 'w', newline='')
csv_new_writer = csv.writer(csv_new_file)
csv_new_writer.writerow(['Frame', 'Landmark Name', 'Landmark ID', 'X', 'Y', 'Z'])



# Video capture for the new video
cap_new = cv2.VideoCapture(new_video_path)

while cap_new.isOpened():
    ret, frame = cap_new.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        for idx, (landmark_id, landmark_name) in enumerate(zip(landmark_indices, landmark_names)):
            landmark = landmarks[landmark_id]
            h, w, c = frame.shape
            cx, cy = int(landmark.x * w), int(landmark.y * h)

            # Save landmark data to CSV
            csv_new_writer.writerow([cap_new.get(cv2.CAP_PROP_POS_FRAMES), landmark_name, idx, landmark.x, landmark.y, landmark.z])

# Close the CSV file for the new data
csv_new_file.close()


# Read the new CSV file into a DataFrame
df_new = pd.read_csv('data_compare.csv')

# Train model
# Extract features and labels
X = df_new[['X', 'Y', 'Z']]
y = df_new['Landmark ID']


# Extract features for the new data
X_new = df_new[['X', 'Y', 'Z']]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Standardize features for the new data
X_scaled_new = scaler.transform(X_new)

# Predict landmark IDs using the trained model
predictions_new = model.predict(X_scaled_new)

# Add predictions to the new DataFrame
df_new['Predicted Landmark ID'] = predictions_new.astype(int)



# Calculate and display the differences per landmark name
df_diff = df_new[['Landmark Name', 'Landmark ID', 'Predicted Landmark ID']]
df_diff['Difference'] = df_diff['Landmark ID'] - df_diff['Predicted Landmark ID']

     
    
    
    # Calculate and display mean difference in percentage for each landmark
mean_differences = df_diff.groupby('Landmark Name')['Difference'].mean()
mean_landmark_values = df_diff.groupby('Landmark Name')['Landmark ID'].mean()
percentage_mean_differences = (mean_differences / mean_landmark_values.abs()) * 100




# Create a list of dictionaries from the printed information
data_list = []
for landmark_name, percentage_difference in percentage_mean_differences.items():
    data_list.append({
        'Landmark Name': landmark_name,
        'Mean Difference Percentage': percentage_difference
    })

# Create a new DataFrame from the list of dictionaries
df_mean_differences = pd.DataFrame(data_list)

# Sort the DataFrame by the 'Mean Difference Percentage' column in descending order
df_mean_differences = df_mean_differences.sort_values(by='Mean Difference Percentage', ascending=False)


# Save the new DataFrame to a CSV file
df_mean_differences.to_csv('Data_for_comp.csv', index=False)
print("New DataFrame saved to 'Data_compare_from_print.csv'")






print("\nMean Difference in Percentage for Each Landmark:")
for landmark_name, percentage_difference in percentage_mean_differences.items():
    print(f"{landmark_name}: {percentage_difference:.2f}%")
    
    
    
    
    

# Filter out inf values before calculating overall mean difference
non_inf_differences = percentage_mean_differences.replace([np.inf, -np.inf], np.nan).dropna()
overall_mean_difference = non_inf_differences.mean()

print(f"\nOverall Mean Difference in Percentage: {overall_mean_difference:.2f}%")

New DataFrame saved to 'Data_compare_from_print.csv'

Mean Difference in Percentage for Each Landmark:
LEFT_ANKLE: 29.66%
LEFT_ELBOW: 41.89%
LEFT_HIP: 46.55%
LEFT_KNEE: 20.38%
LEFT_SHOULDER: 46.19%
LEFT_WRIST: 30.72%
NOSE: -inf%
RIGHT_ANKLE: -6.34%
RIGHT_ELBOW: -112.43%
RIGHT_HIP: 18.74%
RIGHT_KNEE: -23.25%
RIGHT_SHOULDER: -112.43%
RIGHT_WRIST: -78.58%

Overall Mean Difference in Percentage: -8.24%


C:\Users\pmarques\AppData\Local\Temp\ipykernel_14188\2142381812.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diff['Difference'] = df_diff['Landmark ID'] - df_diff['Predicted Landmark ID']


In [4]:
# Filter out inf values before calculating overall mean difference
non_inf_differences = percentage_mean_differences.replace([np.inf, -np.inf], np.nan).dropna()
overall_mean_difference = non_inf_differences.mean()

print(f"\nOverall Mean Difference in Percentage: {overall_mean_difference:.2f}%")


Overall Mean Difference in Percentage: -0.94%


In [6]:
df_comp= pd.read_csv('Data_for_comp.csv')
df_comp


,Landmark Name,Mean Difference Percentage
0,LEFT_ELBOW,52.047059
1,LEFT_SHOULDER,37.117647
2,LEFT_WRIST,36.470588
3,LEFT_HIP,31.058824
4,LEFT_ANKLE,16.470588
5,RIGHT_HIP,14.319328
6,LEFT_KNEE,13.240642
7,RIGHT_ANKLE,-1.202614
8,RIGHT_KNEE,-3.411765
9,RIGHT_WRIST,-30.901961
